[View in Colaboratory](https://colab.research.google.com/github/CC-MNNIT/2018-19-Classes/blob/master/MachineLearning/2018_08_27_Logical-Rhythm-3/MNIST.ipynb)

# MNIST Tutorial By [packetChor](https://github.com/packetChor) उर्फ Dipunj Gupta

1.   Download the MNIST dataset from Professor Yann Lecun's Website
2.   Create a folder called DATA
3. Extract the files (which are in idx format)



In [ ]:
!wget -q -r -A '*ubyte.gz' --no-parent  'http://yann.lecun.com/exdb/mnist/'
!rm -rf DATA
!mkdir DATA
!mv yann.lecun.com/exdb/mnist/* DATA/
!rm -rf yann.lecun.com
!gunzip DATA/*

In [ ]:
!ls DATA

To convert idx directly to an array (numpy array), use idx2numpy library, pydot,graphviz is used for visualising the model's structure

In [ ]:
!pip3 install -q idx2numpy
!pip3 install -q --upgrade --force-reinstall matplotlib

In [ ]:
import idx2numpy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
X_train = idx2numpy.convert_from_file('DATA/train-images-idx3-ubyte')
Y_train = idx2numpy.convert_from_file('DATA/train-labels-idx1-ubyte')

X_test = idx2numpy.convert_from_file('DATA/t10k-images-idx3-ubyte')
Y_test = idx2numpy.convert_from_file('DATA/t10k-labels-idx1-ubyte')

In [ ]:
# Creating a Cross Validation Set from the training data

X_cross = X_train[50000:]
Y_cross = Y_train[50000:]

X_train = X_train[:50000]
Y_train = Y_train[:50000]

In [ ]:
print("X_Train",X_train.shape)
print("Y_Train",X_train.shape)

print("X_cross",X_cross.shape)
print("Y_cross",X_cross.shape)

print("X_Test",X_test.shape)
print("Y_Test",X_test.shape)

The images are stored as 2D arrays/matrix (of size 28*28) where each value represents the intensity of a pixel.

In [ ]:
np.set_printoptions(linewidth=np.nan)
print(X_train[12])


# Can you see THE BIG 3 in the image below ??

In [ ]:
digit = X_train[12]
plt.imshow(digit)
plt.show()


# plt.imshow() treats 0 as brightest and 255 as darkest

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils

In [ ]:
X_train = X_train.reshape(X_train.shape[0], -1) # flattens X_train to 50,000 * (1*28*28) : Basically keeps the first dimension intact and flattens all other dimensions
X_train = X_train.astype('float32')
X_train /= 255

X_cross = X_cross.reshape(X_cross.shape[0], -1)
X_cross = X_cross.astype('float32')
X_cross /= 255

X_test = X_test.reshape(X_test.shape[0], -1)
X_test = X_test.astype('float32')
X_test /= 255


Y_train = np_utils.to_categorical(Y_train, 10)
Y_cross = np_utils.to_categorical(Y_cross, 10)
Y_test = np_utils.to_categorical(Y_test, 10)

num_classes = Y_test.shape[1]

In [ ]:
print("X_Train",X_train.shape)
print("Y_Train",X_train.shape)

print("X_cross",X_cross.shape)
print("Y_cross",X_cross.shape)

print("X_Test",X_test.shape)
print("Y_Test",X_test.shape)

In [ ]:
model = Sequential()

# First Hidden Layer (input layer is implicit from the input_dim)
model.add( Dense(units=100, input_shape=(28*28,), activation='relu',name='Hidden-1'))

# Second Hidden Layer
model.add( Dense(units=100, activation='relu',name='Hidden-2') )

# Output layer
model.add( Dense(units=num_classes, activation='sigmoid',name='OutputLayer') )

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# sgd stands for : stochastic gradient descent (the same gradient descent that we taught you in class)

In [ ]:
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
from IPython.display import Image

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
Image(filename='model_plot.png') 

In [ ]:
history = model.fit(X_train, Y_train, validation_data=(X_cross, Y_cross), epochs=20, batch_size=200, verbose=1)

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'cross validation'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'cross validation'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test accuracy:', score[1])

In [ ]:
Y_pred = model.predict_classes(X_test,verbose=1)
Y_test = np.argmax(Y_test, axis=1) 

In [ ]:
correct_indices = list(np.where(Y_pred == Y_test)[0])
incorrect_indices = list(np.where(Y_pred != Y_test)[0])

In [ ]:
plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')

for i, correct in enumerate(correct_indices[:81]):
    plt.subplot(9,9,i+1)
    plt.imshow(X_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Prediction = {}, Actual = {}".format(Y_pred[correct], Y_test[correct]))   

plt.tight_layout()

In [ ]:
plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')

for i, incorrect in enumerate(incorrect_indices[:81]):
    plt.subplot(9,9,i+1)
    plt.imshow(X_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Prediction = {}, Actual = {}".format(Y_pred[incorrect], Y_test[incorrect]))   

plt.tight_layout()